In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install openai
!pip install pandas

In [2]:
import os 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openai import OpenAI
from timeout_decorator import timeout

In [3]:
def get_data_from_url():
    """
    Function that opens Vigoshop.si URL and parses data.
    """
    
    url = 'https://vigoshop.si'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        ul_element = soup.find('ul', class_='products columns-4')

        # Extract information from li elements within the ul
        if ul_element:
            product_list = ul_element.find_all('li', class_='product')

        return product_list
    else:
        print('Napaka pri pridobivanju spletne strani. Status code:', response.status_code)
        return None


def generate_response(client, messages):
    """
    Call OpenAI ChatGPT (so far we will use GPT-3.5-turbo).
    :param client: OpenAI object
    :param messages: array of message rules
    :return response 
    """
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0,
      max_tokens=256
    )
    return response


def get_data(online=True): 
    """
    Function that reads data from URL and saves it or reads saved data. 
    :param online: bool, whether we want to use real data or data from URL. 
    :return pd.DataFrame
    """
    filename = "products.csv"
    if online: 
        df = list()
        data = get_data_from_url()

        if data != None:
            for product in data:
                anchor = product.find('a')
                href = anchor.get('href')
                img = product.find('img')
                img_url = img.get('src')
                text = product.text.split("\n")[:3]
                price = text[2][:-1].replace(",", ".")
                name = text[1]
                df.append([name, price, href, img_url])

        column_names = ["name", "price", "href", "img_url"]
        df = pd.DataFrame(df, columns=column_names)
        df['img_url'] = df['img_url'].str.replace('_extract', '')
        df["additional_name"] = df.name
        #df.set_index("name", inplace=True, drop=True)
        df.to_csv(filename)
    else: 
        df = pd.read_csv(filename)
    return df


In [4]:
def prepare_input_messages(item_name, target_group, must_use_words, language="slovenian"):
    # Prepare prompt messages to guide your ChatBot
    rules_slovenian = ""
    rules_english = "You are a helpful assistant for creating marketing ads on Facebook platform. You will use all knowladge to set up the most clickable ad." + \
                    "You know that facebook allows ad headers of length 40 characters and contrent of length 125 characters." + \
                    "Answer should be formed as header in one line and content in second line. So it shoul have only 2 lines! First one with mamimum 40 characters and second with maximum 125 characters. " + \
                    f"Prepare response in {language} language"

    messages=[
        {"role": "system", "content": rules_english},
        ]
    if target_group != "": 
        target_group_message = f" Create an ad to target {target_group}"
    if must_use_words != "": 
        must_use_words_message = f" You must use the following words in ad text: '{must_use_words}'"

    additional_message = {"role": "assistant", "content": target_group_message + must_use_words_message}
    messages.append({"role": "user", "content": f"Create facebook add for: {item_name} (the name is in slovenian). THe header should not be item name. "
                                                 + target_group_message 
                                                 + must_use_words_message
                                                 +"You know that facebook allows ad headers of length 40 characters and contrent of length 125 characters."
                    }
                   )

    return messages


def create_ad(client, name, target_group, must_use_words): 
    messages = prepare_input_messages(name, target_group, must_use_words)
    generated_text = generate_response(client, messages)
    text = generated_text.choices[0].message.content
    return text
    

In [5]:
df = get_data(online=True)
client = OpenAI()

target_group = "single people"
must_use_words = ""

for index, row in df.head(10).iterrows(): 
    name = row["name"]
    try: 
        ad = create_ad(client, name, target_group, must_use_words)
    except TimeoutError:
        print("ERROR")
    print(ad)
    print("--------------------")
    break
    time.sleep(30)
        

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
df.loc[32]

In [ ]:
df.head(1).values

In [ ]:
from timeout_decorator import timeout

In [ ]:
!pip install timeout_decorator